In [1]:
import numpy as np
from src import generate_file, read_file, read_mm_file
from pathlib import Path
import multiprocessing as mp
from multiprocessing import Process, Pool, Manager

In [2]:
FILE_SIZE_GB = 2
FILE_PATH = Path('./output.npy')
#generate_file(FILE_SIZE_GB)

# Последовательное чтение
Выполнение операций в одном процессе

С использованием функций numpy

In [5]:
%%time
data_ = read_file()
sum_, min_, max_ = np.sum(data_), np.min(data_), np.max(data_)
print(f' sum: {sum_} \n min: {min_} \n max:{max_}')

 sum: 1152924073519233306 
 min: 5 
 max:4294967295
CPU times: user 2.69 s, sys: 2.78 s, total: 5.47 s
Wall time: 19.1 s


С использованием стандартных функций

In [15]:
%%time
data_ = read_file()
sum_, min_, max_ = sum(data_), min(data_), max(data_)
print(f' sum: {sum_} \n min: {min_} \n max:{max_}')

 sum: 1152924073519233306 
 min: 5 
 max:4294967295
CPU times: user 2min 42s, sys: 0 ns, total: 2min 42s
Wall time: 2min 48s


# Параллельное чтение

Рассмотрены подходы: 
- с распараллеливанием по задачам / по данным
- с использованием mmap-файла, определенного с помощью модуля mmap стандартной библиотеки / с помощью numpy

Вариант с параллелизацией по задачам (толком не понял, как это сделать через mp.Pool, поэтому такой вариант):

In [6]:
def task_parallelism(data):
    def sum_job(arr, q):
        q.put(np.sum(arr))

    def max_job(arr, q):
        q.put(np.max(arr))

    def min_job(arr, q):
        q.put(np.min(arr))

    q = mp.Queue()
    jobs = (sum_job, max_job, min_job)
    args = ((data, q), (data, q), (data, q))

    for job, arg in zip(jobs, args):
        mp.Process(target=job, args=arg).start()

    for i in range(len(jobs)):
        print(f'Result of {jobs[i].__name__}: {q.get()}')

In [7]:
%%time
custom_mmap = read_mm_file()
task_parallelism(custom_mmap)

Result of sum_job: 5
Result of max_job: 1152924073519233306
Result of min_job: 4294967295
CPU times: user 2.62 ms, sys: 211 ms, total: 213 ms
Wall time: 5.57 s


In [8]:
%%time
dt = np.dtype(np.uint32).newbyteorder('big')
np_mmap = np.memmap('./output.npy', dtype=dt, mode='r') 
task_parallelism(np_mmap)

Result of sum_job: 4294967295
Result of max_job: 1152924073519233306
Result of min_job: 5
CPU times: user 0 ns, sys: 193 ms, total: 193 ms
Wall time: 1.37 s


Вариант с параллелизацией по данным

In [9]:
def get_res(array):
    return np.sum(array), np.min(array), np.max(array)

In [10]:
def data_parallelism(data):
    with Pool(4) as pool:
        res = np.array(pool.map(get_res, data))
    return np.sum(res[:, 0]), np.min(res[:, 1]), np.max(res[:, 2])

In [13]:
%%time
custom_mmap = read_mm_file()
custom_mmap_split = np.array_split(custom_mmap, 4)
data_parallelism(custom_mmap_split)

CPU times: user 1.1 s, sys: 3.17 s, total: 4.27 s
Wall time: 14.1 s


(1152924073519233306, 5, 4294967295)

In [14]:
%%time
dt = np.dtype(np.uint32).newbyteorder('big')
np_mmap = np.memmap('./output.npy', dtype=dt, mode='r') 
np_mmap_split = np.array_split(np_mmap, 4)
data_parallelism(np_mmap_split)

CPU times: user 1.01 s, sys: 2.87 s, total: 3.88 s
Wall time: 11.4 s


(1152924073519233306, 5, 4294967295)

При последовательном подходе, как и следовало ожидать, оптимальными оказались функции numpy.
При параллельном подходе лучше всего себя показал метод с распараллеливанием по задачам + np.mmap. 